In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

# Set pandas display options to show all rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


In [ ]:
import numpy as np
from segysak.segy import (
    get_segy_texthead,
    segy_header_scan,
    segy_header_scrape,
)
import xarray as xr

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from dask.distributed import Client

client = Client()
client

In [ ]:
import pathlib

V3D_path = pathlib.Path(r"data\01_raw\VOLVE\seismic\ST10010\Stacks\ST10010ZC11_PZ_PSDM_RAW_FULL_T.MIG_RAW.POST_STACK.3D.JS-017536.segy")
seisnc = xr.open_dataset(
    V3D_path,
    # engine="netcdf4",
    dim_byte_fields={"iline": 189, "xline": 193},
    extra_byte_fields={"cdp_x": 181, "cdp_y": 185},
)

seisnc.segysak.scale_coords()

print(seisnc.seis.humanbytes)
print(seisnc.chunks)

In [ ]:
get_segy_texthead(V3D_path)

In [ ]:
import pandas as pd

well_survey = pd.read_csv(r"C:\projects\quick_pp\notebooks\data\01_raw\VOLVE\VOLVE_15-9_Surveys.csv")

well_survey.head()

In [ ]:
from quick_pp.geophysics.seismic import extract_seismic_along_well

# Extract seismic along well trajectory
# First prepare well coordinates dataframe
well_coords = well_survey[well_survey['WELL_NAME'] == '15-9-19-A'][['MD (m RKB)', 'UTM E/W (m)', 'UTM N/S (m)', 'TVD (m RKB)']]
well_coords.columns = ['md', 'x', 'y', 'z']

In [ ]:
from quick_pp.geophysics.seismic import plot_well_trajectory

fig = plot_well_trajectory(seisnc, well_coords)

In [ ]:
from quick_pp.geophysics.seismic import plot_seismic_along_well

fig = plot_seismic_along_well(seisnc, well_coords)

In [ ]:
import quick_pp.las_handler as las

with open(r'data\01_raw\VOLVE\15-9-19-A.las', 'rb') as f:
    df, header = las.read_las_files([f])

In [ ]:
from quick_pp.geophysics.seismic import *
layers = auto_identify_layers(df.GR, df.RHOB, df.DT, df.DEPTH)
plt.figure(figsize=(15, 3))
plt.plot(layers)


In [ ]:
ref = calculate_reflectivity(df[['DEPTH', 'GR', 'RHOB', 'DT']])
ref.plot(x='DEPTH', y='REFLECTIVITY')

In [ ]:
well_seismic = extract_seismic_along_well(seisnc, well_coords)
well_trace_df = well_seismic.to_dataframe().reset_index()
well_trace_df['TRACE'] = well_trace_df['data']
ref2 = pd.merge_asof(ref, well_trace_df, on='DEPTH')
W = optimize_wavelet(ref2.TRACE, ref2.REFLECTIVITY.fillna(0))

In [ ]:
ref2.head()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(15, 5), sharex=True)
axs[0].plot(W[0], label='Wavelet')
axs[0].legend()
axs[1].plot(ref2.TRACE, label='Seismic')
axs[1].legend()
axs[2].plot(ref2.REFLECTIVITY, label='Reflectivity')
axs[2].legend()